In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import re
import plotly.express as px


#### Import Data Sheets
- Metadata = df
- Area counts = AC

In [2]:
df = pd.read_excel('Data/Unlinked Metadata 0625.xlsx')
df.head()

,Subject ID,Patient Consented,Sample Type_#,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,HMF Y/N?,TPN Y/N?,Iron Y/N?,Linked?,# Aliquots,Additional Comments
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,Y,NaN,NaN,N,5 (~0.8),Residual from Milk Prep Room & B2 Full
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,Y,N,Y,N,2,Scavenged Feeding Tube
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,Y,N,Y,N,1,Scavenged Feeding Tube
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,Y,N,Y,N,1,Scavenged Feeding Tube
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,Y,Y,NaN,N,7,Residual from Milk Prep Room


In [3]:
df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace from column names

In [4]:
df.columns

Index(['Subject ID', 'Patient Consented', 'Sample Type_#', 'CGA', 'DOL',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N?', 'Linked?',
       '# Aliquots', 'Additional Comments'],
      dtype='object')

In [5]:
AC = pd.read_excel('Data/NeoBank Unlinked AC.xlsx')
AC.head()

,sample ID,inj vol,2'FL,DFLAC,3'SL,6'SL,LNT,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00003_M_27,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NB00003_M_26,15,0.149068,0.025539,0.036240,0.048252,0.032744,0.015477,0.037367,0.029173,0.014387,0.029336,0.024442,0.024050,0.014162,0.024403,0.014275
2,NB00010_M_4,15,3.202351,0.211063,0.202066,0.793208,1.288569,0.099609,0.896822,0.838056,0.026299,0.198385,0.816358,0.293375,0.064694,0.116699,0.158850
3,NB00004_M_4,15,4.533413,0.015644,0.109186,0.516448,0.978176,0.235237,2.135244,0.025100,0.028135,0.273544,0.066877,0.159376,0.057803,0.033015,0.069969
4,NB00014_M_1,15,6.616323,0.997031,0.286334,0.743593,1.071591,0.337355,2.218069,0.367415,0.051010,0.659550,0.795395,0.261531,0.038317,0.048243,0.148319


In [6]:
AC.columns

Index(['sample ID', 'inj vol', '2'FL', 'DFLAC', '3'SL', '6'SL', 'LNT', 'LNnT',
       'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc', 'DFLNT', 'DSLNT', 'DFLNH',
       'FDSLNH', 'DSLNH'],
      dtype='object')

In [7]:
# rename columns without the "'"
AC = AC.rename(columns={"2'FL": "2FL", "3'SL": "3SL", "6'SL": "6SL"})

In [8]:
# Rename columns for consistency

df = df.rename(columns={"Sample Type_#": "sample_unique_id"})
AC = AC.rename(columns={"sample ID": "sample_unique_id"})


In [9]:
df["sample_unique_id"] = df["sample_unique_id"].astype(str).str.strip()  # Ensure sample_unique_id is a string and strip whitespace
AC["sample_unique_id"] = AC["sample_unique_id"].astype(str).str.strip()

#### Merge the Metadata and Area Counts DataFrames
- inner join = samples that exist in both DataFrames

In [10]:
merged = pd.merge(df, AC, on="sample_unique_id", how="inner")

In [11]:
merged

,Subject ID,Patient Consented,sample_unique_id,CGA,DOL,Current Weight,Current Height,Current HC,Scavenged/Fresh?,MBM/DMB?,...,LNnT,LNFPI,LNFPII,LNFPIII,LSTc,DFLNT,DSLNT,DFLNH,FDSLNH,DSLNH
0,NB00012,N,NB00012_M_1,28.4,27,1430,32.0,26.0,Scavenged,MBM + DBM,...,0.089623,0.470256,1.205369,0.014793,0.239572,0.281999,0.173245,0.215138,0.348744,0.384665
1,NB00003,Y,NB00003_M_1,29.0,22,995,35.7,23.7,Scavenged,DBM,...,0.054806,0.337448,0.352980,0.011570,0.075870,0.375191,0.066067,0.044498,0.038670,0.034095
2,NB00003,Y,NB00003_M_2,29.0,22,1005,36.0,24.4,Scavenged,DBM,...,0.113728,0.508929,0.384073,0.009342,0.082319,0.296118,0.063142,0.073261,0.057761,0.057419
3,NB00003,Y,NB00003_M_3,29.3,25,1065,36.0,24.3,Scavenged,DBM,...,0.078861,0.579639,0.381617,0.012011,0.080428,0.181844,0.061159,0.052973,0.090722,0.052794
4,NB00354,N,NB00354_M_1,29.4,12,1005,34.0,34.5,Scavenged,MBM,...,0.049258,0.108794,1.039603,0.024323,0.258750,0.138556,0.164200,0.203099,0.342770,0.364438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,NB00007,Y,NB00007_M_2,38.4,12,2190,44.0,32.5,Scavenged,MBM,...,0.197148,1.031503,0.691480,0.038184,0.353092,0.681753,0.194094,0.063247,0.092453,0.154003
124,NB00007,Y,NB00007_M_3,39.2,17,2340,44.0,33.0,Scavenged,MBM,...,0.093871,0.826393,0.565923,0.023937,0.160066,0.732057,0.248158,0.140775,0.036514,0.117773
125,NB00007,Y,NB00007_M_4,39.3,18,2380,44.0,33.0,Scavenged,MBM,...,0.062626,0.250724,1.749928,0.062896,0.362996,0.159926,0.227344,0.067247,0.204275,0.226578
126,NB00379,Y,NB00379_M_1,39.6,98,3073,45.0,33.2,Scavenged,MBM,...,0.074441,1.492769,0.211561,0.009555,0.160130,0.005741,0.066606,0.061999,0.013559,0.146605


In [12]:
print(merged.shape) #number of rows is the number of samples that exist in both DataFrames, 127

(128, 32)


In [13]:
merged.columns

Index(['Subject ID', 'Patient Consented', 'sample_unique_id', 'CGA', 'DOL',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N?', 'Linked?',
       '# Aliquots', 'Additional Comments', 'inj vol', '2FL', 'DFLAC', '3SL',
       '6SL', 'LNT', 'LNnT', 'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc', 'DFLNT',
       'DSLNT', 'DFLNH', 'FDSLNH', 'DSLNH'],
      dtype='object')

#### Sample Inconsistencies?

In [14]:
#list of IDs in the metadata but not the AC file

meta_only = set(df['sample_unique_id']) - set(AC['sample_unique_id'])
print(f"Number of samples in metadata but not in area counts: {len(meta_only)}")

Number of samples in metadata but not in area counts: 7


In [15]:
#list of IDs in the AC file but not in the metadata

AC_only = set(AC['sample_unique_id']) - set(df['sample_unique_id'])
print(f"Number of samples in area counts but not in metadata: {len(AC_only)}")

Number of samples in area counts but not in metadata: 7


In [16]:
print(f"IDs in Metadata Only: {meta_only}")
print(f"IDs in Area Counts Only: {AC_only}")

IDs in Metadata Only: {'NB00003_M_5', 'NB00423_M_1', 'NB00003_M_24', 'NB00002_M_7', 'NB00007_M_7', 'NB00003_M_25', 'NB00001_M_8'}
IDs in Area Counts Only: {'NB00002_M_9 ?', 'NB00008_M_6', 'NB00001_M_2', 'NB00101_M_1', 'NB00361_M_6', 'NB00007_M_6', 'NB00361_M_7'}


### Clean Contents of Merged DataFrame

In [17]:
# Remove all spelling and capitalization inconsistencies in the 'Scavenged/Fresh?' column

merged["Scavenged/Fresh?"].value_counts()

Scavenged/Fresh?
Scavenged     109
Scavenged      18
scavenged       1
Name: count, dtype: int64

In [18]:
def normalize_scavenged(value):
    if isinstance(value, str) and value.strip().lower().startswith('scavenged'):
        return 'Scavenged'
    return value

merged["Scavenged/Fresh?"] = merged["Scavenged/Fresh?"].apply(normalize_scavenged)

In [19]:
merged["Scavenged/Fresh?"].value_counts()

Scavenged/Fresh?
Scavenged    128
Name: count, dtype: int64

In [20]:
merged["MBM/DMB?"].value_counts()

MBM/DMB?
MBM                       88
DBM                       21
Switched to Fortifier      9
MBM                        4
MBM + DBM                  2
FBM/MBM                    1
Name: count, dtype: int64

In [21]:
 # Remove all spelling and capitalization inconsistencies in the 'MBM/DMB?' column

def code_mbm(value):
    if isinstance(value, str) and value.strip().lower().replace(' ', '') == 'mbm':
        return 'MBM'
    return value

merged["MBM/DMB?"] = merged["MBM/DMB?"].apply(code_mbm)

In [22]:
merged["MBM/DMB?"].value_counts()

MBM/DMB?
MBM                       92
DBM                       21
Switched to Fortifier      9
MBM + DBM                  2
FBM/MBM                    1
Name: count, dtype: int64

In [23]:
merged["Iron Y/N?"].value_counts()

Iron Y/N?
Y    48
N    25
Name: count, dtype: int64

In [24]:
# Normalize the aliquots column to extract numeric values

def extract_numeric_aliquots(value):
    match = re.match(r'^\d+', str(value))
    return int(match.group()) if match else None

merged['Aliquots_num'] = merged['# Aliquots'].apply(extract_numeric_aliquots)

In [25]:
merged["Aliquots_num"].value_counts()

Aliquots_num
1     54
7     12
5     10
2      9
3      9
9      8
4      7
8      6
6      6
10     3
13     2
14     1
11     1
Name: count, dtype: int64

In [26]:
merged.columns

Index(['Subject ID', 'Patient Consented', 'sample_unique_id', 'CGA', 'DOL',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N?', 'Linked?',
       '# Aliquots', 'Additional Comments', 'inj vol', '2FL', 'DFLAC', '3SL',
       '6SL', 'LNT', 'LNnT', 'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc', 'DFLNT',
       'DSLNT', 'DFLNH', 'FDSLNH', 'DSLNH', 'Aliquots_num'],
      dtype='object')

#### Check concentration distribution for 2'FL - UPDATE WITH CORRECTED AC FRAME

In [27]:
merged["2FL"].describe()

count    126.000000
mean       2.668561
std        2.210266
min        0.034985
25%        0.300394
50%        2.904018
75%        4.311703
max        8.572130
Name: 2FL, dtype: float64

In [28]:
fig = px.histogram(merged, x="2FL", nbins=40, title='2fl Concentration Distribution')
fig

In [29]:
# Classify secretor status using 5M AUC cutoff
merged["Secretor Status"] = merged["2FL"].apply(
    lambda x: "Secretor" if x >= .5 else "Non-Secretor"
)

In [30]:
merged["Secretor Status"].value_counts()

Secretor Status
Secretor        81
Non-Secretor    47
Name: count, dtype: int64

In [31]:
merged.columns

Index(['Subject ID', 'Patient Consented', 'sample_unique_id', 'CGA', 'DOL',
       'Current Weight', 'Current Height', 'Current HC', 'Scavenged/Fresh?',
       'MBM/DMB?', 'HMF Y/N?', 'TPN Y/N?', 'Iron Y/N?', 'Linked?',
       '# Aliquots', 'Additional Comments', 'inj vol', '2FL', 'DFLAC', '3SL',
       '6SL', 'LNT', 'LNnT', 'LNFPI', 'LNFPII', 'LNFPIII', 'LSTc', 'DFLNT',
       'DSLNT', 'DFLNH', 'FDSLNH', 'DSLNH', 'Aliquots_num', 'Secretor Status'],
      dtype='object')

#### Additional Comments Cleaning

In [32]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                                       68
Scavenged Feeding Tube                                                             48
Scavenged Feeding Tube + NaCl Oral Solution                                         2
Residual from Milk Prep Room & B2 Full                                              1
Scavenged Feeding Tube - Nurse needed to feed extra 1 mL                            1
Scavenged Feeding Tube - also started on sodium chloride oral solution 6/15/24      1
Scanvenged Feeding Tube - Collected Outside of 4 hr Window                          1
Scavenged Feeding Tube + NaCl Oral Solution - Pt intubated 6/25 & is NPO            1
Scavenged Feeding Tube - Outside 4-hr .indo. by 2 hours                             1
Scavenged Feeding Tube - Intubated 6/25 - NPO, restarted NG feeds 7/2               1
Scavenged Feeding Tube - Parenteral Nutrition Stopped                               1
Scavenged Feeding Tube - Last NG F

In [33]:
# Extract scavenged notes from the 'Additional Comments' column

def extract_scavenged_notes(comment):
    if isinstance(comment, str) and "scavenged feeding tube" in comment.lower():
        # Look for + or - and any text after it
        match = re.search(r"(scavenged feeding tube.*?)([+-].*)", comment, re.IGNORECASE)
        if match:
            return match.group(2).strip()
    return None

merged["scavenged notes"] = merged["Additional Comments"].apply(extract_scavenged_notes)

In [34]:
merged["scavenged notes"].value_counts()

scavenged notes
+ NaCl Oral Solution                                       2
- Nurse needed to feed extra 1 mL                          1
- also started on sodium chloride oral solution 6/15/24    1
+ NaCl Oral Solution - Pt intubated 6/25 & is NPO          1
- Outside 4-hr .indo. by 2 hours                           1
- Intubated 6/25 - NPO, restarted NG feeds 7/2             1
- Parenteral Nutrition Stopped                             1
- Last NG Feed 7/2                                         1
Name: count, dtype: int64

In [35]:
# Remove leading '+', '-', or spaces from 'scavenged notes'
merged["scavenged notes"] = merged["scavenged notes"].str.lstrip('+- ').replace('', None)

In [36]:
def clean_scavenged_feeding_tube(comment):
    if isinstance(comment, str):
        # Find 'scavenged feeding tube' and remove everything after it (including +, -, or any text)
        match = re.search(r"(scavenged feeding tube)", comment, re.IGNORECASE)
        if match:
            return comment[:match.end()].strip()
    return comment

merged["Additional Comments"] = merged["Additional Comments"].apply(clean_scavenged_feeding_tube)

In [37]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                            68
Scavenged Feeding Tube                                                  57
Residual from Milk Prep Room & B2 Full                                   1
Scanvenged Feeding Tube - Collected Outside of 4 hr Window               1
Residual from Milk Prep Room - Baby not on NG Feeds no linked sample     1
Name: count, dtype: int64

In [38]:
# Update "Residual from Milk Prep Room & B2 Full" to "Residual from Milk Prep Room" in 'Additional Comments'
merged["Additional Comments"] = merged["Additional Comments"].replace(
    "Residual from Milk Prep Room & B2 Full", "Residual from Milk Prep Room"
)

In [39]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                            69
Scavenged Feeding Tube                                                  57
Scanvenged Feeding Tube - Collected Outside of 4 hr Window               1
Residual from Milk Prep Room - Baby not on NG Feeds no linked sample     1
Name: count, dtype: int64

In [40]:
merged["Additional Comments"] = merged["Additional Comments"].replace(
    "Scanvenged Feeding Tube - Collected Outside of 4 hr Window ", "Scavenged Feeding Tube"
)

In [41]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room                                            69
Scavenged Feeding Tube                                                  58
Residual from Milk Prep Room - Baby not on NG Feeds no linked sample     1
Name: count, dtype: int64

In [42]:
merged["Additional Comments"] = merged["Additional Comments"].replace(
    "Residual from Milk Prep Room - Baby not on NG Feeds no linked sample", "Residual from Milk Prep Room"
)

In [43]:
merged["Additional Comments"].value_counts()

Additional Comments
Residual from Milk Prep Room    70
Scavenged Feeding Tube          58
Name: count, dtype: int64

In [44]:
merged["Additional Comments"] = merged["Additional Comments"].replace(
    "Residual from Milk Prep Room", "Prepped in Milk Room"
)

In [45]:
merged["Additional Comments"].value_counts()

Additional Comments
Prepped in Milk Room      70
Scavenged Feeding Tube    58
Name: count, dtype: int64

In [46]:
merged[["Additional Comments", "scavenged notes"]]

,Additional Comments,scavenged notes
0,Prepped in Milk Room,None
1,Scavenged Feeding Tube,None
2,Scavenged Feeding Tube,None
3,Scavenged Feeding Tube,None
4,Prepped in Milk Room,None
...,...,...
123,Prepped in Milk Room,None
124,Prepped in Milk Room,None
125,Prepped in Milk Room,None
126,Prepped in Milk Room,None


In [47]:
merged["scavenged notes"].value_counts()

scavenged notes
NaCl Oral Solution                                       2
Nurse needed to feed extra 1 mL                          1
also started on sodium chloride oral solution 6/15/24    1
NaCl Oral Solution - Pt intubated 6/25 & is NPO          1
Outside 4-hr .indo. by 2 hours                           1
Intubated 6/25 - NPO, restarted NG feeds 7/2             1
Parenteral Nutrition Stopped                             1
Last NG Feed 7/2                                         1
Name: count, dtype: int64

In [48]:
merged["Additional Comments"].value_counts()

Additional Comments
Prepped in Milk Room      70
Scavenged Feeding Tube    58
Name: count, dtype: int64

In [49]:
def classify_sample_method(comment):
    if isinstance(comment, str):
        if "scavenged" in comment.lower():
            return "Scavenged"
        elif "milk room" in comment.lower():
            return "Prepped in Milk Room"
    return "Other"

merged["Sample Source"] = merged["Additional Comments"].apply(classify_sample_method)


In [50]:
merged["Sample Source"].value_counts()

Sample Source
Prepped in Milk Room    70
Scavenged               58
Name: count, dtype: int64

#### want to make a new column with the mother's secretor status for each sample based on the first MBM sample for each subject

In [51]:
# Make sure columns are clean
merged.columns = merged.columns.str.strip()

# Step 1. Filter to only MBM rows
merged_mbm = merged[merged["MBM/DMB?"] == "MBM"].copy()

# Step 2. Get the *first MBM row* per subject
first_mbm = (
   merged_mbm.sort_values(["Subject ID", "DOL"])   # sort by day of life so "first" makes sense
          .groupby("Subject ID", as_index=False)
          .first()[["Subject ID", "Secretor Status"]]
)

# Step 3. Rename the column
first_mbm.rename(columns={"Secretor Status": "secretorstatus_mom"}, inplace=True)

# Step 4. Merge back to main df
merged = merged.merge(first_mbm, on="Subject ID", how="left")

In [52]:
merged["secretorstatus_mom"] = merged["secretorstatus_mom"].fillna("No MBM")

In [53]:
merged[["Subject ID", "MBM/DMB?", "Secretor Status", "secretorstatus_mom"]]

,Subject ID,MBM/DMB?,Secretor Status,secretorstatus_mom
0,NB00012,MBM + DBM,Secretor,No MBM
1,NB00003,DBM,Secretor,No MBM
2,NB00003,DBM,Secretor,No MBM
3,NB00003,DBM,Secretor,No MBM
4,NB00354,MBM,Non-Secretor,Non-Secretor
...,...,...,...,...
123,NB00007,MBM,Secretor,Secretor
124,NB00007,MBM,Secretor,Secretor
125,NB00007,MBM,Non-Secretor,Secretor
126,NB00379,MBM,Secretor,Secretor


In [54]:
merged.secretorstatus_mom.value_counts()

secretorstatus_mom
Secretor        57
Non-Secretor    42
No MBM          29
Name: count, dtype: int64

In [55]:
### per unique subject id
results = []
for subj, group in merged_mbm.groupby("Subject ID"):
    group = group.sort_values("DOL")
    first_status = group.iloc[0]["Secretor Status"]
    changed = (group["Secretor Status"] != first_status).any()
    
    results.append({
        "Subject ID": subj,
        "secretorstatus_mom": first_status,
        "Change Flag": "Changed" if changed else "No Change"
    })

# ✅ This table has ONE ROW per subject
status_check = pd.DataFrame(results)

print("Number of unique subjects in summary:", status_check["Subject ID"].nunique())
status_check


Number of unique subjects in summary: 28


,Subject ID,secretorstatus_mom,Change Flag
0,NB00001,Non-Secretor,No Change
1,NB00002,Non-Secretor,Changed
2,NB00004,Secretor,No Change
3,NB00005,Secretor,No Change
4,NB00006,Secretor,No Change
5,NB00007,Secretor,Changed
6,NB00008,Secretor,No Change
7,NB00009,Secretor,No Change
8,NB00010,Secretor,No Change
9,NB00011,Secretor,Changed


##### Download Status_Check as a DF

In [56]:
status_check.to_excel('Secretor_status_check.xlsx', index=False)

### Download the cleaned and merged DataFrame

In [ ]:
merged.columns

In [ ]:
merged.to_excel('Unlinked_Merged.xlsx', index=False)

In [ ]:
merged

### is HMO concentration accosicated with metadata features?

In [ ]:
pip install scipy


In [ ]:
merged

## Secretor Status via Type of Milk

In [ ]:
mbm_non = merged[
    (merged["MBM/DMB?"] == "MBM") & 
    (merged["Secretor Status"] == "Non-Secretor")
]

count = len(mbm_non)
count_non = mbm_non["Subject ID"].nunique()
count_non

print(f"Number of MBM samples from Non-Secretors: {count}")
print(f"Number of unique Non-Secretor subjects with MBM samples: {count_non}")

In [ ]:
mbm_sec = merged[
    (merged["MBM/DMB?"] == "MBM") & 
    (merged["Secretor Status"] == "Secretor")
]

count_s = len(mbm_sec)
count_sec = mbm_sec["Subject ID"].nunique()

print(f"Number of MBM samples fromn Secretors: {count_s}")
print(f"Number of unique Secretor subjects with MBM samples: {count_sec}")

In [ ]:
per_non_mbm = count_non/(count_non + count_s)*100
print(f"Percent of Non-Secretor for MBM samples: {per_non_mbm:2f}%")

In [ ]:
mbm_non = merged[
    (merged["MBM/DMB?"] == "MBM") & 
    (merged["Secretor Status"] == "Non-Secretor")
]

count = len(mbm_non)
count_non = mbm_non["sample_unique_id"].nunique()
count_non

In [ ]:
dbm_non = merged[
    (merged["MBM/DMB?"] == "DBM") & 
    (merged["Secretor Status"] == "Non-Secretor")
]

count_d = len(dbm_non)
count_dnon = dbm_non["Subject ID"].nunique()

print(f"Number of DBM samples from Non-Secretors: {count_d}")
print(f"Number of unique Non-Secretor subjects with DBM samples: {count_dnon}")

In [ ]:
dbm_sec = merged[
    (merged["MBM/DMB?"] == "DBM") & 
    (merged["Secretor Status"] == "Secretor")
]

count_d_s = len(dbm_sec)
count_dsec = dbm_sec["Subject ID"].nunique()

print(f"Number of DBM samples from Secretors: {count_d_s}")
print(f"Number of unique Secretor subjects with DBM samples: {count_dsec}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

sns.boxplot(data=merged, x="TPN Y/N?", y="2FL")
plt.title("2’FL Concentration by TPN Use")
plt.show()

# Mann-Whitney U test (non-parametric)
group1 = merged[merged["TPN Y/N?"] == "Y"]["2FL"].dropna()
group2 = merged[merged["TPN Y/N?"] == "N"]["2FL"].dropna()

stat, p = mannwhitneyu(group1, group2)
print(f"Mann-Whitney U test: U={stat:.2f}, p={p:.4f}")


### Only with Subject IDs with more than 3 timepoints

In [ ]:
subject_counts = merged["Subject ID"].value_counts()
longitudinal_subjects = subject_counts[subject_counts > 3].index

filtered_df = merged[merged["Subject ID"].isin(longitudinal_subjects)].copy()

In [ ]:
filtered_df.columns = filtered_df.columns.str.strip()


In [ ]:
filtered_df["MBM/DMB?"]

In [ ]:
filtered_df.columns

In [ ]:
df = merged.rename(columns=lambda x: x.strip())  # Remove trailing spaces
df["DOL"] = pd.to_numeric(df["DOL"], errors="coerce")

In [ ]:
import plotly.express as px

fig = px.line(
    df[df["Subject ID"] == 'NB00003'],  # set selected_subject as needed
    x="DOL",
    y="2FL",
    color="TPN Y/N?",  # or "TPN Y/N?"
    markers=True,
    title=f"2’FL over Time for {'NB00003'} by HMF Status"
)
fig.update_layout(
    plot_bgcolor="#1E1E1E",
    paper_bgcolor="#1E1E1E",
    font_color="white"
)
fig.show()


In [ ]:
from scipy.stats import mannwhitneyu

group1 = df[df["TPN Y/N?"] == "Y"]["2FL"].dropna()
group2 = df[df["TPN Y/N?"] == "N"]["2FL"].dropna()

stat, p = mannwhitneyu(group1, group2)
print(f"TPN Effect on 2’FL: U = {stat:.2f}, p = {p:.4f}")


In [ ]:
df.columns

In [ ]:
df["hover_label"] = (
    "TPN: " + df["TPN Y/N?"].astype(str) + "<br>" +
    "HMF: " + df["HMF Y/N?"].astype(str) + "<br>" +
    "Iron: " + df["Iron Y/N"].astype(str) + "<br>" +
    "Scavenged: " + df["Scavenged/Fresh?"].astype(str) + "<br>" +
    "Notes: " + df["Additional Comments"].astype(str)
)

In [ ]:
import plotly.express as px

In [ ]:
df["Iron Size"] = df["Iron Y/N"].map({"Y": 20, "N": 10})


In [ ]:
import plotly.express as px

fig = px.scatter(
    df[df["Subject ID"] == "NB00003"],
    x="DOL",
    y="2FL",
    color="TPN Y/N?",
    symbol="Scavenged/Fresh?",
    size="Iron Size",
    hover_name="DOL",
    hover_data={
        "TPN Y/N?": True,
        "HMF Y/N?": True,
        "Iron Y/N": True,
        "Scavenged/Fresh?": True,
        "Additional Comments": True
    },
    text="HMF Y/N?",
    title="2’FL over Time for NB00003"
)


fig.update_layout(
    plot_bgcolor="#1E1E1E",
    paper_bgcolor="#1E1E1E",
    font_color="white"
)

fig.show()


## Type of Milk Collected

In [ ]:
merged["Additional Comments"].value_counts()

In [ ]:
filtered_merged = merged[
    merged["Additional Comments"].isin(["Residual from Milk Prep Room", "Scavenged Feeding Tube"])
].copy()

In [ ]:
filtered_merged

In [ ]:
def classify_sample_method(comment):
    if isinstance(comment, str):
        if "scavenged feeding tube" in comment.lower():
            return "Scavenged"
        elif "milk prep room" in comment.lower():
            return "Residual"
    return "Other"

filtered_merged["Sample Source"] = filtered_merged["Additional Comments"].apply(classify_sample_method)


In [ ]:
filtered_merged["Sample Source"].value_counts()

In [ ]:
filtered_merged["CGA"].describe()

In [ ]:
filtered_merged["CGA Bin"] = pd.cut(
    filtered_merged["CGA"], 
    bins=[28, 32, 34, 36, 41], 
    labels=["Very Preterm (<32)", "Moderate Preterm (32-34)", "Late Preterm (34–36)", "Term (35–40)"]
)
